### Final Project
#### Submitted by:  1. Ebenezer Daniel  2. Raja Prabhakaran 3. Nitheesh Samiappan
-- change at the end, add some format

Dataset details: -- fill this

In [1]:
import importlib
import subprocess
import sys

# List of required libraries
required_libraries = [
    "pandas", "numpy", "matplotlib", "seaborn", "scipy",
    "nltk", "scikit-learn", "scikit-surprise"  # Use scikit-surprise instead of surprise
]

# Function to check and install missing libraries
def check_and_install(libraries):
    for lib in libraries:
        try:
            importlib.import_module(lib)
        except ImportError:
            print(f"Installing {lib}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
            except subprocess.CalledProcessError:
                print(f"Failed to install {lib}. Trying Conda (if applicable)...")
                try:
                    subprocess.check_call(["conda", "install", "-c", "conda-forge", lib, "-y"])
                except subprocess.CalledProcessError:
                    print(f"Could not install {lib}. Please install it manually.")

# Check and install missing libraries
check_and_install(required_libraries)

print("All required libraries are installed.")


Installing scikit-learn...
Installing scikit-surprise...
All required libraries are installed.


In [21]:
# Jupyter magic command for inline plots
%matplotlib inline

# Import libraries
import pandas as pd
import glob
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings
warnings.simplefilter('ignore')

print("All required libraries are successfully imported.")

All required libraries are successfully imported.


In [3]:
# Define the folder where your CSV files are stored
csv_folder = "Datasets/" 

# Use glob to get only CSV files that start with 'book'
csv_book_files = glob.glob(os.path.join(csv_folder, "book*.csv"))
csv_user_files = glob.glob(os.path.join(csv_folder, "user*.csv"))

# Load and concatenate all "book" CSV files into a single DataFrame
# Function to read CSV files and standardize column names before combining
def load_and_standardize_csv(file_list):
    df_list = []
    for file in file_list:
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# Load and combine "book" and "user" CSV files
df_books = load_and_standardize_csv(csv_book_files)
df_users = load_and_standardize_csv(csv_user_files)

# Rename specific mismatched columns for consistency
df_books.rename(columns={'pagesnumber': 'pagesNumber'}, inplace=True)

print(f"Combined 'book' DataFrame shape: {df_books.shape}")
print(f"Combined 'user' DataFrame shape: {df_users.shape}")

Combined 'book' DataFrame shape: (1850310, 20)
Combined 'user' DataFrame shape: (362596, 3)


In [4]:
#store the merged data into a csv files for both books and user
df_books.to_csv('combined_books_data.csv', index=False) # Causes memory error
df_users.to_csv('combined_users_data.csv', index=False)

In [5]:
#copy the data frame and keep the original data for further use
df_books_original = df_books.copy()
df_users_original = df_users.copy()
print('Original data copied for future use. DONE!!!')

Original data copied for future use. DONE!!!


### Preprocessing & Data Cleaning

In [6]:
# find the unique ID and ISBN
unique_ids = df_books['id'].nunique()
unique_isbns = df_books['isbn'].nunique()
print(f"Unique IDs: {unique_ids}")
print(f"Unique ISBNs: {unique_isbns}")

Unique IDs: 1850115
Unique ISBNs: 1844192


In [7]:
# find duplicates across all rows
duplicate_rows = df_books.duplicated().sum()
print(f"Duplicate Rows: {duplicate_rows}")

Duplicate Rows: 112


In [8]:
# find missing ISBNs & ID
missing_isbns = df_books['isbn'].isna().sum()
print(f"Missing ISBNs: {missing_isbns}")

missing_id = df_books['id'].isna().sum()
print(f"Missing Id: {missing_id}")

Missing ISBNs: 5923
Missing Id: 0


In [9]:
# Drop duplicate rows
df_books_cleaned = df_books.drop_duplicates()
print(f"After removing fully duplicated rows: {df_books_cleaned.shape[0]} records remaining.")

# Keep only the first occurrence of each ID
df_books_cleaned = df_books_cleaned.drop_duplicates(subset=['id'], keep='first')
print(f"After ensuring unique IDs: {df_books_cleaned.shape[0]} records remaining.")

After removing fully duplicated rows: 1850198 records remaining.
After ensuring unique IDs: 1850115 records remaining.


In [10]:
# Check for missing (NaN) values across all columns
missing_values = df_books_cleaned.isna().sum()
missing_values

id                             0
name                           0
ratingdist1                    0
pagesNumber                    0
ratingdist4                    0
ratingdisttotal                0
publishmonth                   0
publishday                     0
publisher                  17821
countsofreview                 0
publishyear                    0
language                 1598369
authors                        0
rating                         0
ratingdist2                    0
ratingdist5                    0
isbn                        5922
ratingdist3                    0
description               678927
count of text reviews    1440418
dtype: int64

In [11]:
# Dropping all the unnesscary columns
df_books_cleaned.drop(columns=['count of text reviews', 'isbn'])
df_books_cleaned.head(5)

,id,name,ratingdist1,pagesNumber,ratingdist4,ratingdisttotal,publishmonth,publishday,publisher,countsofreview,publishyear,language,authors,rating,ratingdist2,ratingdist5,isbn,ratingdist3,description,count of text reviews
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960,NaN,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781,NaN,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,2:130310,5:4268227,NaN,3:567458,NaN,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,2:49353,5:1504505,0439554896,3:288821,NaN,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,2:24849,5:1749958,043965548X,3:194848,NaN,NaN


In [12]:
# from summa import summarizer

# # Function to summarize book descriptions
# def summarize_description(row, df):
#     if pd.isna(row['Description']):
#         similar_books = df[df['Authors'] == row['Authors']]['Description'].dropna()
#         if len(similar_books) > 0:
#             text = " ".join(similar_books[:5])  # Take descriptions from similar books
#             return summarizer.summarize(text, words=50)  # Generate a short summary
#     return row['Description']

# # Apply function to dataset
# df_books_cleaned['Description'] = df_books_cleaned.apply(lambda row: summarize_description(row, df_books_cleaned), axis=1)

# # Fill remaining missing values
# df_books_cleaned['Description'].fillna("No description available.", inplace=True)


In [13]:
# find different types of rating in the rating colum
ratings = df_users['rating'].unique()
print(ratings)

['it was amazing' 'really liked it' 'liked it' 'did not like it'
 'it was ok' "This user doesn't have any rating"]


In [14]:
# Ask Prof for -->This user doesn't have any rating, can we remove them or assign 0

In [15]:
rating_mapping = {
    "it was amazing": 5,
    "really liked it": 4,
    "liked it": 3,
    "it was ok": 2,
    "did not like it": 1,
    "This user doesn't have any rating": None  # Convert to NaN or remove
}

# create a numeric rating colum
df_users['rating in numbers'] = df_users['rating'].map(rating_mapping)
df_users.head(5)

,id,name,rating,rating in numbers
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5.0
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5.0
2,1,Siddhartha,it was amazing,5.0
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4.0
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4.0


In [16]:
df_users_copy = df_users.copy()
df_books_copy = df_books_cleaned.copy() 

In [17]:
df_books_copy.rename(columns={'id': 'book_id'}, inplace=True)
df_users_copy.rename(columns={'id': 'user_id'}, inplace=True)

In [18]:
df_users_copy.head()

,user_id,name,rating,rating in numbers
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5.0
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5.0
2,1,Siddhartha,it was amazing,5.0
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4.0
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4.0


In [19]:
df_books_copy.head()

,book_id,name,ratingdist1,pagesNumber,ratingdist4,ratingdisttotal,publishmonth,publishday,publisher,countsofreview,publishyear,language,authors,rating,ratingdist2,ratingdist5,isbn,ratingdist3,description,count of text reviews
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960,NaN,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781,NaN,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,2:130310,5:4268227,NaN,3:567458,NaN,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,2:49353,5:1504505,0439554896,3:288821,NaN,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,2:24849,5:1749958,043965548X,3:194848,NaN,NaN


In [27]:
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df_users_copy['clean_name'] = df_users_copy['name'].apply(clean_text)
df_books_copy['clean_name'] = df_books_copy['name'].apply(clean_text)

In [28]:
name_to_book_id_map = df_books_copy.set_index('clean_name')['book_id'].to_dict()
df_users_copy['book_id'] = df_users_copy['clean_name'].map(name_to_book_id_map)

In [29]:
matched_count = df_users_copy['book_id'].notna().sum()
print(f"Directly matched count: {matched_count}")

Directly matched count: 234734


In [30]:
unmatched_count = df_users_copy['book_id'].isna().sum()
print(f"Directly not matched count: {unmatched_count}")

Directly not matched count: 127862


In [31]:
# First, map directly where names match exactly (FAST)
df_users_copy['book_id'] = df_users_copy['clean_name'].map(name_to_book_id_map)

# Count directly matched entries
matched_count = df_users_copy['book_id'].notna().sum()
print(f"Directly matched count: {matched_count}")


Directly matched count: 234734


In [32]:
df_users_copy.head(20)

,user_id,name,rating,rating in numbers,clean_name,book_id
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5.0,agile web development with rails a pragmatic g...,45.0
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5.0,the restaurant at the end of the universe hitc...,862825.0
2,1,Siddhartha,it was amazing,5.0,siddhartha,828548.0
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4.0,the clock of the long now time and responsibility,1788479.0
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4.0,ready player one ready player one 1,NaN
5,1,"The Hunger Games (The Hunger Games, #1)",it was amazing,5.0,the hunger games the hunger games 1,3544003.0
6,1,"The Clue in the Embers (Hardy Boys, #35)",it was amazing,5.0,the clue in the embers hardy boys 35,239732.0
7,1,The Authoritative Calvin and Hobbes: A Calvin ...,it was amazing,5.0,the authoritative calvin and hobbes a calvin a...,502937.0
8,1,"The Clue of the Broken Blade (Hardy Boys, #21)",it was amazing,5.0,the clue of the broken blade hardy boys 21,76933.0
9,1,"The Clue of the Hissing Serpent (Hardy Boys, #53)",it was amazing,5.0,the clue of the hissing serpent hardy boys 53,NaN


In [33]:
unmatched_rows = df_users_copy[df_users_copy['book_id'].isna()]
unmatched_rows.to_csv("unmatched_books.csv", index=False)

In [34]:
matched_rows = df_users_copy[df_users_copy['book_id'].notna()]
matched_rows.to_csv("matched_books.csv", index=False)

In [35]:
unique_book_ids = df_users_copy['book_id'].nunique()
print(f"Total unique book IDs: {unique_book_ids}")

Total unique book IDs: 51336
